# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Datas important

1. Ler o arquivo de regiões metropolitanas e equipes esportivas associadas em `assets/wikipedia_data.html` e responder a algumas perguntas sobre cada região metropolitana. 

2. Cada uma dessas regiões pode ter uma ou mais equipes das "Big 4": NFL (futebol americano, em `assets/nfl.csv`), MLB (beisebol, em `assets/mlb.csv`), NBA (basquete, em `assets/nba.csv`) ou NHL (hóquei, em `assets/nhl.csv`). 

3. Exige-se alguma compreensão humana dos dados fora dos dados que você recebeu.

4. Quer-se a *correlação* entre a proporção de *vitórias/derrotas* e a *população da cidade* em que está localizada. A proporção de vitórias/derrotas refere-se ao número de vitórias dividido pelo número total de vitórias mais o número de derrotas, i.e, $\operatorname{ratio\_win\_loss}_i = \frac{\# \ \text{win}_i}{\# \ \text{win}_i \ + \ \# \ \text{loss}_i}, \quad i = 1, 2, 3, \cdots, n \ (\text{total teams})$.

5. Para obter a correlação de Pearson, você vai enviar duas listas ordenadas de valores, as populações do arquivo wikipedia_data.html` e a proporção de vitórias/derrotas para um determinado esporte na mesma ordem.

6. Fazer a média das proporções de vitórias/derrotas para aquelas cidades que têm várias equipes de um único esporte.

7. Usar apenas dados do ano de 2018 para a sua análise.

8. Não usar dados de MLS ou CFL.

9. Pode haver mais equipes do que os testes de declarações assertivas, lembre-se de consolidar várias equipes em uma única cidade em um único valor. 

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
# library 
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# datas
cities_df = pd.read_html("assets/wikipedia_data.html")[1]
nhl = pd.read_csv("assets/nhl.csv")

# rename columns 
cities_df.rename({'Metropolitan area': 'metropolitan_area', 'Country': 'country', 
                'Population (2016 est.)[8]': 'population_2016'}, axis = 1, inplace = True)

# excluir row and change type of column population
cities_df = cities_df.iloc[:-1, :]
cities_df['population_2016'] = np.float64(cities_df['population_2016'])

# functions
def test_convert(element):
    try:
        result = int(element)
    except:
        result = element
    return type(result) == int

def nhl_correlation(): 
    
    # datas
    nhl_df = nhl.copy()
    city_df = cities_df.copy()
    
    # text mining, filter and explode cities
    city_df = city_df.iloc[:, [0, 3, 8]] 
    city_df.iloc[:, -1] = city_df.iloc[:, -1].replace("\[[\w\s]*\]|[—]", value = "", regex = True).apply(lambda x: re.split("\s", x))
    city_df = city_df.explode('NHL', ignore_index = True)
    city_df = city_df[~(city_df['NHL'] == "")]

    
    # filter
    nhl_df = nhl_df[(nhl_df['year'] == 2018) & 
                    (nhl_df['League'] == 'NHL') & 
                    (nhl_df['W'].apply(test_convert))]
    
    nhl_df = nhl_df.iloc[:, [0, 2, 3]]
    
    # text mining
    nhl_df['team'].replace(to_replace = "\*$", value = "", regex = True, inplace = True)
    nhl_df['team'] = nhl_df['team'].apply(lambda x: re.split("\s", x))
    nhl_df = nhl_df.explode('team', ignore_index = True)
    
    # new data frame
    metropolitan_team_df = city_df.merge(right = nhl_df, left_on = 'NHL', right_on = 'team', how = 'inner', sort = True)
    
    # change type of column
    metropolitan_team_df[['W', 'L']] = metropolitan_team_df[['W', 'L']].astype(np.float64)
    metropolitan_team_df["ratio_win_loss"] = metropolitan_team_df['W'] / (metropolitan_team_df['W'] + metropolitan_team_df['L'])
    metropolitan_team_df = metropolitan_team_df.groupby('metropolitan_area').agg({'ratio_win_loss': np.nanmean, 
                                                       'population_2016': np.nanmean})
    
    # final variables
    population_by_region = metropolitan_team_df['population_2016'] # pass in metropolitan area population from cities
    win_loss_by_region = metropolitan_team_df['ratio_win_loss'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
nba = pd.read_csv("assets/nba.csv")


def nba_correlation():
    
    # data
    nba_df = nba.copy()
    city_df = cities_df.copy().iloc[:,[0,3,7]]   
    
    # mining text
    nba_df['team'].replace(to_replace = '[\*]?\s+\(\w+\)', value = "", regex = True, inplace = True)
    city_df.iloc[:, -1] = city_df.iloc[:, -1].replace("\[[\w\s]*\]|[—]", value = "", regex = True).apply(lambda x: re.split("\s", x))
    
    # columns types
    nba_df = nba_df[['team', 'W', 'L', 'year']]
    
    # filter for year 2018, columns team, W, L, and year remain
    nba_df = nba_df[(nba_df['year'] == 2018) & (nba_df['W'].apply(test_convert))]
         
    # explode
    nba_df['team'] = nba_df['team'].apply(lambda x: re.split('\s', x))
    nba_df = nba_df.explode('team', ignore_index = True)
    city_df = city_df.explode('NBA', ignore_index = True)
    city_df = city_df[~(city_df['NBA'] == "")]
    
    # new data frame
    metropolitan_team_df = city_df.merge(right = nba_df, left_on = 'NBA', right_on = 'team', how = 'inner', sort = True)
    
    # change type of column
    metropolitan_team_df[['W', 'L']] = metropolitan_team_df[['W', 'L']].astype(np.float64)
    metropolitan_team_df["ratio_win_loss"] = metropolitan_team_df['W'] / (metropolitan_team_df['W'] + metropolitan_team_df['L'])
    metropolitan_team_df = metropolitan_team_df.groupby('metropolitan_area').agg({'ratio_win_loss': np.mean, 
                                                       'population_2016': np.max})
    
    # final variables
    population_by_region = metropolitan_team_df['population_2016'] # pass in metropolitan area population from cities
    win_loss_by_region = metropolitan_team_df['ratio_win_loss'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def gets_area(team):
    for each in list(mlb_cities.index.values):
        if  team in each: return mlb_cities.at[each, 'Metropolitan area']
        
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

population =cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population['Metropolitan area']=population['Metropolitan area'].str.strip()
population.rename(columns={'Population (2016 est.)[8]':'Population'},inplace=True)
population = population.set_index('Metropolitan area')

cities['MLB'].replace(r'(.*)\[.*\].*|[-—]',r'\1',regex=True, inplace=True)
cities['MLB'].replace('—', np.nan, inplace=True)
cities['MLB']=cities['MLB'].str.strip('—').str.strip(' ')
cities['MLB'].replace('', np.nan, inplace=True)
cities.dropna(inplace=True)

mlb_cities = cities[['Metropolitan area', 'MLB']].set_index('MLB')

mlb_df=mlb_df[mlb_df['year']==2018].drop(['GB','W-L%','League','year'],axis=1)

mlb_df['team'].replace(r'(.*)[\*]|[\(].*',r"\1".strip(),regex=True, inplace=True)
mlb_df[['team']]=mlb_df[['team']].apply(lambda x: x.str.strip())
mlb_df['Area']=mlb_df['team'].str.split(" ").str[-1:]
mlb_df["Area"]= mlb_df["Area"].str[0]
mlb_df['Area'] = mlb_df['Area'].apply(lambda x: gets_area(x))

mlb_df.loc[0,'Area']='Boston'
mlb_df[['W','L']] = mlb_df[['W','L']].apply(pd.to_numeric, axis=1)
mlb_df=mlb_df.groupby('Area').sum()
mlb_df.eval("Ratio =W / (W + L)", inplace=True)
mlb_df.drop(['W','L'], axis=1,inplace=True)

out_df = pd.merge(mlb_df, population, how="inner", left_index=True, right_index=True)
out_df['Population']=pd.to_numeric(out_df['Population'])

def mlb_correlation(): 
    
    population_by_region = out_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region =out_df['Ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
nfl = pd.read_csv("assets/nfl.csv")

def nfl_correlation(): 

    # pre-processin cities
    city_df = cities_df.iloc[:, [0,3,5]]
    city_df['NFL'] = city_df['NFL'].replace('\[[\w\s]+\]|[—]', '', regex = True).apply(lambda x: re.split("\s", x))
    city_df = city_df.explode('NFL', ignore_index = True)
    city_df = city_df[~(city_df['NFL'] == "")]

    # pre_processing nfl
    nfl_df = nfl.copy()
    nfl_df = (nfl_df[['team', 'W', 'L', 'year']])[(nfl_df['year'] == 2018) & nfl_df['W'].apply(lambda x: test_convert(x))]
    nfl_df['team'] = (nfl_df['team'].replace('[\*\+]$', value = '', regex = True)).apply(lambda x: re.split('\s', x))
    nfl_df = nfl_df.explode('team', ignore_index = True)
    nfl_df[['W', 'L']] =  nfl_df[['W', 'L']].astype(np.float64)

    # new data frame
    metropolitan_team_df = city_df.merge(right = nfl_df, left_on = 'NFL', right_on = 'team', how = 'inner', sort = True)
    
    # change type of column
    metropolitan_team_df[['W', 'L']] = metropolitan_team_df[['W', 'L']].astype(np.float64)
    metropolitan_team_df["ratio_win_loss"] = metropolitan_team_df['W'] / (metropolitan_team_df['W'] + metropolitan_team_df['L'])
    metropolitan_team_df = metropolitan_team_df.groupby('metropolitan_area').agg({'ratio_win_loss': np.mean, 
                                                       'population_2016': np.max})
    
    # final variables
    population_by_region = metropolitan_team_df['population_2016'] # pass in metropolitan area population from cities
    win_loss_by_region = metropolitan_team_df['ratio_win_loss'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

1. Ho: **dado que uma área possui duas equipes esportivas em esportes diferentes, essas equipes terão o mesmo desempenho em seus respectivos esportes**

2. Usar uma série de testes t pareados (portanto, use `ttest_rel`) entre todos os pares de esportes. 

3. Existem esportes nos quais podemos rejeitar a hipótese nula? 

4. Novamente, faça a média dos valores onde um esporte tem várias equipes em uma região. 

5. Lembre-se, você só incluirá, para cada esporte, cidades que têm equipes envolvidas nesse esporte, descartando outras conforme apropriado. Esta pergunta vale 20% da nota para esta tarefa.

In [5]:
cities_df = pd.read_html("assets/wikipedia_data.html")[1]
cities_df = cities_df.iloc[:-1,[0,3,5,6,7,8]]
 
def sports_team_performance():
    
    city_df = cities_df
    city_df.rename({'Metropolitan area':'metropolitan_area',
                    'Population (2016 est.)[8]':'population_2016'}, axis = 1, inplace = True)

    city_df['NFL'] = city_df['NFL'].replace('\[[\w\s]+\]|(—\s?)', '', regex = True).apply(lambda x: re.split("\s", x))
    city_df['MLB'] = city_df['MLB'].replace('\[[\w\s]+\]|[—]', '', regex = True).apply(lambda x: re.split("\s", x))
    city_df['NBA'] = city_df['NBA'].replace('\[[\w\s]+\]|[—]', '', regex = True).apply(lambda x: re.split("\s", x))
    city_df['NHL'] = city_df['NHL'].replace('\[[\w\s]+\]|[—]', '', regex = True).apply(lambda x: re.split("\s", x))  
    city_df['count_sport'] = city_df[['NFL', 'MLB', 'NBA', 'NHL']].apply(lambda x: sum([(0 if i == [''] else 1) for i in x]), axis = 1)
    city_df = city_df[city_df['count_sport'] >= 2]
    city_df.drop('count_sport', axis = 1, inplace = True)

    # pre=processing league sport
    mlb_df = pd.read_csv("assets/mlb.csv")
    nhl_df = pd.read_csv("assets/nhl.csv")
    nba_df = pd.read_csv("assets/nba.csv")
    nfl_df = pd.read_csv("assets/nfl.csv")
    
        # nhl
    nhl_df = nhl_df[(nhl_df['year'] == 2018) & (nhl_df['League'] == 'NHL') & (nhl_df['W'].apply(test_convert))]
    nhl_df = nhl_df.iloc[:, [0, 2, 3]]
    nhl_df['team'].replace(to_replace = "\*$", value = "", regex = True, inplace = True)
    nhl_df['team'] = nhl_df['team'].apply(lambda x: re.split("\s", x))
    nhl_df = nhl_df.explode('team', ignore_index = True)
    nhl_df[['W', 'L']] =  nhl_df[['W', 'L']].astype(np.float64)
    nhl_df['ratio_win_loss'] = nhl_df['W'] / nhl_df['W'] + nhl_df['L']
    nhl_df = nhl_df.iloc[:, [0, 3]]

        # nba
    nba_df['team'].replace(to_replace = '[\*]?\s+\(\w+\)', value = "", regex = True, inplace = True)
    nba_df = nba_df[['team', 'W', 'L', 'year']]
    nba_df = nba_df[(nba_df['year'] == 2018) & (nba_df['W'].apply(test_convert))]
    nba_df['team'] = nba_df['team'].apply(lambda x: re.split('\s', x))
    nba_df = nba_df.explode('team', ignore_index = True)
    nba_df[['W', 'L']] =  nba_df[['W', 'L']].astype(np.float64)
    nba_df['ratio_win_loss'] = nba_df['W'] / nba_df['W'] + nba_df['L']
    nba_df = nba_df.iloc[:, [0, 4]]

        # mlb
    mlb_df = mlb[['team', 'year', 'W', 'L']]
    mlb_df = mlb_df[(mlb_df['year'] == 2018) & (mlb_df['W'].apply(lambda x: test_convert(x)))]
    mlb_df[['W', 'L']] = mlb_df[['W', 'L']].astype(np.float64)
    mlb_df['team'] = mlb_df['team'].replace('[\*\+]$', value = '', regex = True).apply(lambda x: re.split('\s', x))
    mlb_df = mlb_df.explode('team', ignore_index = True)
    mlb_df['ratio_win_loss'] = mlb_df['W'] / mlb_df['W'] + mlb_df['L']
    mlb_df = mlb_df.iloc[:, [0, 4]]

        # nfl
    nfl_df = (nfl_df[['team', 'W', 'L', 'year']])[(nfl_df['year'] == 2018) & nfl_df['W'].apply(lambda x: test_convert(x))]
    nfl_df['team'] = (nfl_df['team'].replace('[\*\+]$', value = '', regex = True)).apply(lambda x: re.split('\s', x))
    nfl_df = nfl_df.explode('team', ignore_index = True)
    nfl_df[['W', 'L']] =  nfl_df[['W', 'L']].astype(np.float64)
    nfl_df['ratio_win_loss'] = nfl_df['W'] / nfl_df['W'] + nfl_df['L']
    nfl_df = nfl_df.iloc[:, [0, 4]]

    # explode city for type sport
    city_nhl = city_df[['metropolitan_area', 'NHL']].explode('NHL', ignore_index = True)
    city_nhl = city_nhl[city_nhl['NHL'] != '']
    city_nba = city_df[['metropolitan_area', 'NBA']].explode('NBA', ignore_index = True)
    city_nba = city_nba[city_nba['NBA'] != '']
    city_mlb = city_df[['metropolitan_area', 'MLB']].explode('MLB', ignore_index = True)
    city_mlb = city_mlb[city_mlb['MLB'] != '']
    city_nfl = city_df[['metropolitan_area', 'NFL']].explode('NFL', ignore_index = True)
    city_nfl = city_nfl[city_nfl['NFL'] != '']

    # inner betwen city_sport and sport
    nhl_ratio = nhl_df.merge(right = city_nhl, left_on = 'team', right_on = 'NHL', how = 'inner', sort = True).drop(['team', 'NHL'], axis = 1)
    nba_ratio = nba_df.merge(right = city_nba, left_on = 'team', right_on = 'NBA', how = 'inner', sort = True).drop(['team', 'NBA'], axis = 1)
    mlb_ratio = mlb_df.merge(right = city_mlb, left_on = 'team', right_on = 'MLB', how = 'inner', sort = True).drop(['team', 'MLB'], axis = 1)
    nfl_ratio = nfl_df.merge(right = city_nfl, left_on = 'team', right_on = 'NFL', how = 'inner', sort = True).drop(['team', 'NFL'], axis = 1)

    # mean for metropolitan area 
    nhl_ratio = nhl_ratio.groupby('metropolitan_area')['ratio_win_loss'].agg(np.mean)
    nba_ratio = nba_ratio.groupby('metropolitan_area')['ratio_win_loss'].agg(np.mean)
    mlb_ratio = mlb_ratio.groupby('metropolitan_area')['ratio_win_loss'].agg(np.mean)
    nfl_ratio = nfl_ratio.groupby('metropolitan_area')['ratio_win_loss'].agg(np.mean)

    # inner comparation
    nhl_nba = pd.merge(nhl_ratio, nba_ratio, left_index = True, right_index = True, how = 'inner')
    nhl_mlb = pd.merge(nhl_ratio, mlb_ratio, left_index = True, right_index = True, how = 'inner')
    nhl_nfl = pd.merge(nhl_ratio, nfl_ratio, left_index = True, right_index = True, how = 'inner')
    nba_mlb = pd.merge(nba_ratio, mlb_ratio, left_index = True, right_index = True, how = 'inner')
    nba_nfl = pd.merge(nba_ratio, nfl_ratio, left_index = True, right_index = True, how = 'inner')
    mlb_nfl = pd.merge(mlb_ratio, nfl_ratio, left_index = True, right_index = True, how = 'inner')

    p_value_nfl = [np.nan, 
                   stats.ttest_rel(nba_nfl.iloc[:, 0], nba_nfl.iloc[:, 1])[1], 
                   stats.ttest_rel(nhl_nfl.iloc[:, 0], nhl_nfl.iloc[:, 1])[1],
                   stats.ttest_rel(mlb_nfl.iloc[:, 0], mlb_nfl.iloc[:, 1])[1]]

    p_value_nba = [stats.ttest_rel(nba_nfl.iloc[:, 0], nba_nfl.iloc[:, 1])[1],
                   np.nan,
                   stats.ttest_rel(nhl_nba.iloc[:, 0], nhl_nba.iloc[:, 1])[1],
                   stats.ttest_rel(nba_mlb.iloc[:, 0], nba_mlb.iloc[:, 1])[1]]

    p_value_nhl = [stats.ttest_rel(nhl_nfl.iloc[:, 0], nhl_nfl.iloc[:, 1])[1],
                   stats.ttest_rel(nhl_nba.iloc[:, 0], nhl_nba.iloc[:, 1])[1],
                   np.nan,
                   stats.ttest_rel(nhl_mlb.iloc[:, 0], nhl_mlb.iloc[:, 1])[1]]

    p_value_mlb = [stats.ttest_rel(mlb_nfl.iloc[:, 0], mlb_nfl.iloc[:, 1])[1],
                   stats.ttest_rel(nba_mlb.iloc[:, 0], nba_mlb.iloc[:, 1])[1],
                   stats.ttest_rel(nhl_mlb.iloc[:, 0], nhl_mlb.iloc[:, 1])[1],
                   np.nan]

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    p_values['NFL'] = p_value_nfl
    p_values['NBA'] = p_value_nba
    p_values['NHL'] = p_value_nhl
    p_values['MLB'] = p_value_mlb
    
    # assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    # assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values